In [1]:
from neuron import h, gui
import bokeh as bk
import bokeh.plotting as bkp
import numpy as np
from bokeh.palettes import Dark2_5 as palette
import itertools 
import ipywidgets as widgets
from branched_cable import branched_cable, run_current_clamp

colors = itertools.cycle(palette)  
bkp.output_notebook()

Loading BokehJS ...

In [2]:
def plot_cable(L,diam,Ra,Rm,cm,I,stim_loc):
    # make a cable, insert stimulus halfway down the process
    cell = branched_cable()
    cell.make_cable(L,diam,Ra=Ra,Rm=Rm,cm=cm)
    cell.add_IClamp('cable',stim_loc,.001,50)

    # record at a few different points along the cable
    positions = list(np.linspace(0,1,num=51))
    for pos in positions:   
        cell.recording_vecs([['cable',pos,'v']])

    #print(cell.record_vecs.keys())
    t,results = run_current_clamp(cell,'iclamp',[I],tstop=200)

    RC = bkp.figure(plot_height=400,plot_width=400)
    #keys = list(results.record_vecs.keys())
    for i, color in zip(range(0,len(positions),5),colors):
        key = "cable:%.2f:v" % positions[i]
        RC.line(t,np.squeeze(results[key][:,-1]),color=color,legend=str(positions[i]))
    RC.xaxis.axis_label="I (mA)"
    RC.yaxis.axis_label="mV"


    # steady state V as a function of distance
    ss_ind = np.where(t>95)[0][0]
    V_ss = np.zeros([len(positions),])
    
    V_half = np.zeros([len(positions),])
    tau = np.zeros([len(positions),])
    for i,pos in enumerate(positions):
        #print(pos)
        key = "cable:%.2f:v" % pos
        V_ss[i] = results[key][ss_ind,-1]
        V_half[i] = .5*(V_ss[i]+65)-65
        tau[i] = np.where(results[key][:,-1]>V_half[i])[0][0]/t.size*200. -20
        
        
    VX = bkp.figure(plot_height = 400, plot_width=400)
    VX.line(positions,V_ss)
    VX.xaxis.axis_label = "position"
    VX.yaxis.axis_label = "Steady-state V (mV)"
    
    
    T = bkp.figure(plot_height=400, plot_width=400)
    T.line(positions,tau)
    T.xaxis.axis_label = "position"
    T.yaxis.axis_label = "tau"
    

    bkp.show(bk.layouts.layout([[RC,VX],[T]]))
             

In [3]:
widgets.interact_manual(plot_cable,
                 L=widgets.FloatSlider(min=50,max=1000,continous_update=False),
                 diam=widgets.FloatSlider(min=.1,max=100,continuous_update=False),
                 Rm=widgets.FloatSlider(min=100,max=5000,continuous_update=False),
                 Ra = widgets.FloatSlider(min=10,max=1000,continuous_update=False),
                 cm = widgets.FloatSlider(min=.1,max=10,continous_update=False),
                 I = widgets.FloatSlider(min=.1,max=5,continuous_update=False),
                 stim_loc = widgets.FloatSlider(min=0,max=1,contiuous_update=False))

A Jupyter Widget

<function __main__.plot_cable>

In [4]:

I = np.linspace(-1,1,num=11)
cell = branched_cable()
cell.make_cable(500,1)
cell.add_IClamp('cable',.5,.001,100)

# record at a few different points along the cable
positions = list(np.linspace(0,1,num=51))
keys = []
for pos in positions:   
    keys.append('cable:%.2f:v' % pos)
    cell.recording_vecs([['cable',pos,'v']])

t,results = run_current_clamp(cell,'iclamp',I,tstop=400)

V_ss = np.zeros([I.size,len(keys)])
ss_ind = np.where(t>95)[0][0]
for j,key in enumerate(keys):
    V_ss[:,j] = np.squeeze(results[key][ss_ind,:])
slope = np.squeeze(np.diff(V_ss,axis=0).mean(axis=0))


VI = bkp.figure(plot_height=400,plot_width=400)
for i, color in zip(range(0,int((len(positions)+1)/2),5),colors):
    VI.line(I,V_ss[:,i],color=color)
VI.xaxis.axis_label = "I (mA)"
VI.yaxis.axis_label = "Steady-State V (mV)"
    
S = bkp.figure(plot_height=400,plot_width=400)
S.line(positions[int((len(positions)+1)/2):],slope[int((len(positions)+1)/2):])
S.xaxis.axis_label = "Distance"
S.yaxis.axis_label = "Input Resistance"
bkp.show(bk.layouts.layout([[VI,S]]))


In [21]:
def get_rho(L,diam,Ra,Rm,cm,I):
    cell = branched_cable()
    cell.make_cable(L,diam,Ra=Ra,Rm=Rm,cm=cm)
    cell.add_IClamp('cable',0,.001,100)

    # record at a few different points along the cable
    positions = list(np.linspace(0,1,num=101))
    keys = []
    for pos in positions:   
        keys.append('cable:%.2f:v' % pos)
        cell.recording_vecs([['cable',pos,'v']])

    t,results = run_current_clamp(cell,'iclamp',[I],tstop=400)
    ss_ind = np.where(t>95)[0][0]
    V_half = (1-.37)*(results['cable:0.00:v'][ss_ind,:]+65)-65
    
    V_ss = np.zeros([101,])
    for j,key in enumerate(keys):
        V_ss[j] = np.squeeze(results[key][ss_ind,:])
    
    if I<0:
        rho = np.where(V_ss>=V_half)[0][0]/101.*L
    if I>0:
        rho = np.where(V_ss<=V_half)[0][0]/101.*L
    else:
        rho = np.nan
    return rho

In [22]:
# show rho (length constant) is invariant to current strength
I = np.linspace(-1,1,num=5)
RHO = np.zeros([I.size,])
for i,curr in enumerate(list(I)):
    RHO[i] = get_rho(500,1,300,2800,1,curr)

R = bkp.figure(plot_height = 400,plot_width=400)
R.line(I,RHO)
R.xaxis.axis_label= "I (mA)"
R.yaxis.axis_label = "rho"
bkp.show(R)

In [23]:
# rho as a function of diam
diams = np.linspace(.1,10,num=10)
RHO = np.zeros([diams.size,])
for i,d in enumerate(list(diams)):
    RHO[i] = get_rho(500,d,500,2800,1,1)
    
R = bkp.figure(plot_height = 400,plot_width=400)
R.line(diams,RHO)
R.xaxis.axis_label= "diameter (microns)"
R.yaxis.axis_label = "rho"
bkp.show(R)

In [24]:
# rho as a function of Rm
Rms = np.logspace(2,5,num=10)
RHO = np.zeros([Rms.size,])
for i,Rm in enumerate(list(Rms)):
    RHO[i] = get_rho(500,1,500,Rm,1,1)
    
R = bkp.figure(plot_height = 400,plot_width=400)
R.line(Rms,RHO)
R.xaxis.axis_label= "membrane resistance"
R.yaxis.axis_label = "rho"
bkp.show(R)

In [25]:
# rho as a function of cm
cms = np.linspace(.01,10,num=10)
RHO = np.zeros([cms.size,])
for i,cm in enumerate(list(cms)):
    RHO[i] = get_rho(500,5,300,2800,cm,1)
    
R = bkp.figure(plot_height = 400,plot_width=400)
R.line(cms,RHO)
R.xaxis.axis_label= "Cm"
R.yaxis.axis_label = "rho"
bkp.show(R)

In [26]:
# rho as function of Ra
Ras = np.linspace(300,5000,num=15)
RHO = np.zeros([Ras.size,])
for i,Ra in enumerate(list(Ras)):
    RHO[i] = get_rho(500,5,Ra,2800,1,1)
    
R = bkp.figure(plot_height = 400,plot_width=400)
R.line(Ras,RHO)
R.xaxis.axis_label= "axial resistance"
R.yaxis.axis_label = "rho"
bkp.show(R)

In [4]:
def plot_branched_cable(branch_pos,L,diam,I):
    # make a cable, insert stimulus halfway down the process
    cell = branched_cable()
    cell.make_cable(500,5,Ra=300)
    cell.add_branch('cable',branch_pos,L,diam,Ra=300)
    cell.add_IClamp('cable',0,1,50)

    # record along entire length of cable and one point on the branch
    positions = np.linspace(0,1,num=101)
    for pos in positions:
        cell.recording_vecs([['cable',pos,'v'],['branch',pos,'v']])
        
    #cell.recording_vecs([['branch',.5,'v']])

    #print(cell.record_vecs.keys())
    t,results = run_current_clamp(cell,'iclamp',[I],tstop=200)

    RC = bkp.figure(plot_height=400,plot_width=400)
    #keys = list(results.record_vecs.keys())
    for i, color in zip(range(0,len(positions),10),colors):
        key = "cable:%.2f:v" % positions[i]
        RC.line(t,np.squeeze(results[key][:,-1]),color=color,legend=str(positions[i]))
    RC.line(t,np.squeeze(results['branch:0.50:v'][:,-1]),line_dash='dotted',legend='branch')
    RC.xaxis.axis_label="I (mA)"
    RC.yaxis.axis_label="mV"


    # steady state V as a function of distance
    ss_ind = np.where(t>95)[0][0]
    V_ss = np.zeros([len(positions),])
    V_ss_br = np.zeros([len(positions),])
    
    V_half = np.zeros([len(positions),])
    tau = np.zeros([len(positions),])
    for i,pos in enumerate(positions):
        #print(pos)
        key = "cable:%.2f:v" % pos
        V_ss[i] = results[key][ss_ind,-1]
        
        br_key = "branch:%.2f:v" % pos
        V_ss_br[i] = results[br_key][ss_ind,-1]
        
        V_half[i] = (1-.37)*(V_ss[i]+65)-65
        tau[i] = np.where(results[key][:,-1]>V_half[i])[0][0]/t.size*200. -20
        
        
    VX = bkp.figure(plot_height = 400, plot_width=400)
    VX.line(positions*500,V_ss)
    VX.xaxis.axis_label = "position"
    VX.yaxis.axis_label = "Steady-state V (mV)"
    
    VX.line(500*branch_pos + positions*L,V_ss_br,line_color='red',line_dash='dashed')
    
    
    T = bkp.figure(plot_height=400, plot_width=400)
    T.line(positions,tau)
    T.xaxis.axis_label = "position"
    T.yaxis.axis_label = "tau"
    

    bkp.show(bk.layouts.layout([[RC,VX],[T]]))


In [5]:
widgets.interact_manual(plot_branched_cable,
                 branch_pos = widgets.FloatSlider(min=0,max=1,continuosu_update=False),
                 L=widgets.FloatSlider(min=10,max=500,continous_update=False),
                 diam=widgets.FloatSlider(min=.1,max=10,continuous_update=False),
                 I = widgets.FloatSlider(min=.1,max=5,continuous_update=False))

A Jupyter Widget

<function __main__.plot_branched_cable>